# Fraud in Electricity and Gas Consumption #

## Data Cleaning

### Since 2 datasets were provided, we attempt to combine both datasets into 1 on the id columm. ###

In [104]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import spearmanr


seed = 69


In [34]:
invoice_df = pd.read_csv('invoice.csv')
client_df = pd.read_csv('client.csv')

combined_df = pd.merge(client_df, invoice_df, on='id', how='left')



In [35]:
combined_df.head()

,region,date_x,dis,id,catg,target,date_y,tarif_type,counter_number,counter_statue,...,reading_remarque,consommation_level_4,old_index,new_index,months_number,counter_type,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3
0,101,31/12/1994,60,train_Client_0,11,0,24/3/2014,11,1335667.0,0,...,8,0,14302,14384,4,ELEC,1,82,0,0
1,101,31/12/1994,60,train_Client_0,11,0,29/3/2013,11,1335667.0,0,...,6,0,12294,13678,4,ELEC,1,1200,184,0
2,101,31/12/1994,60,train_Client_0,11,0,23/3/2015,11,1335667.0,0,...,8,0,14624,14747,4,ELEC,1,123,0,0
3,101,31/12/1994,60,train_Client_0,11,0,13/7/2015,11,1335667.0,0,...,8,0,14747,14849,4,ELEC,1,102,0,0
4,101,31/12/1994,60,train_Client_0,11,0,17/11/2016,11,1335667.0,0,...,9,0,15066,15638,12,ELEC,1,572,0,0


In [36]:
combined_df.describe()


,region,dis,catg,target,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,consommation_level_4,old_index,new_index,months_number,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3
count,500651.000000,500651.000000,500651.000000,500651.000000,500651.000000,5.006510e+05,500651.000000,500651.000000,500651.000000,500651.000000,5.006510e+05,5.006510e+05,500651.000000,500651.000000,500651.000000,500651.000000,500651.000000
mean,204.746922,63.519156,11.353871,0.062644,16.108279,1.951034e+11,0.050217,204.390755,7.463710,64.393150,1.575969e+04,1.639037e+04,22.744289,1.000154,443.065463,120.508706,28.196772
std,104.620488,3.388720,3.661420,0.242323,11.145881,2.071552e+12,0.396153,121.204514,1.374409,1230.465569,2.975733e+04,3.053707e+04,1670.624818,0.047150,592.249623,1396.817086,214.020756
min,101.000000,60.000000,11.000000,0.000000,9.000000,0.000000e+00,0.000000,5.000000,6.000000,0.000000,0.000000e+00,0.000000e+00,1.000000,1.000000,0.000000,0.000000,0.000000
25%,101.000000,62.000000,11.000000,0.000000,11.000000,1.477220e+05,0.000000,202.000000,6.000000,0.000000,1.799000e+03,2.165000e+03,4.000000,1.000000,99.000000,0.000000,0.000000
50%,107.000000,62.000000,11.000000,0.000000,11.000000,4.857010e+05,0.000000,203.000000,8.000000,0.000000,7.876000e+03,8.438000e+03,4.000000,1.000000,321.000000,0.000000,0.000000
75%,307.000000,69.000000,11.000000,0.000000,11.000000,1.008740e+06,0.000000,207.000000,9.000000,0.000000,2.092750e+04,2.164500e+04,4.000000,1.000000,661.000000,0.000000,0.000000
max,399.000000,69.000000,51.000000,1.000000,45.000000,2.740000e+13,5.000000,600.000000,9.000000,343568.000000,2.800280e+06,2.870972e+06,231602.000000,20.000000,98889.000000,819886.000000,45360.000000


In [37]:
print(f"Length of dataset: {len(combined_df)}")


print(f"Number of datapoints in each column: \n{combined_df.count()} \n")

Length of dataset: 500651
Number of datapoints in each column: 
region                  500651
date_x                  500651
dis                     500651
id                      500651
catg                    500651
target                  500651
date_y                  500651
tarif_type              500651
counter_number          500651
counter_statue          500651
counter_code            500651
reading_remarque        500651
consommation_level_4    500651
old_index               500651
new_index               500651
months_number           500651
counter_type            500651
counter_coefficient     500651
consommation_level_1    500651
consommation_level_2    500651
consommation_level_3    500651
dtype: int64 



In [38]:
number_of_fraud = sum(combined_df["target"] == 1)
print(number_of_fraud)
print(f"proportion of fraud: {number_of_fraud/len(combined_df)}")

31363
proportion of fraud: 0.06264443694310008


#### We have created 4 new variables, described as such:
##### delta_start_invoice: diff between join and transaction date
##### delta_index: diff between old and new index
##### delta_transactions: diff between transactions over the same client
##### consommation_sum: sum of consommation levels

In [39]:
dates = {'join_date': combined_df['date_x'], 'transaction_date': combined_df['date_y']}
dates_df = pd.DataFrame(dates)

# Assuming 'date_x' and 'date_y' are in 'combined_df'
combined_df['date_x'] = pd.to_datetime(combined_df['date_x'], format='%d/%m/%Y')
combined_df['date_y'] = pd.to_datetime(combined_df['date_y'], format='%d/%m/%Y')

# Calculate the difference in days between 'date_y' and 'date_x'
dates_df['delta_start_invoice'] = (combined_df['date_y'] - combined_df['date_x']).dt.days

# Display the first few rows of the DataFrame
dates_df.head()




,join_date,transaction_date,delta_start_invoice
0,31/12/1994,24/3/2014,7023
1,31/12/1994,29/3/2013,6663
2,31/12/1994,23/3/2015,7387
3,31/12/1994,13/7/2015,7499
4,31/12/1994,17/11/2016,7992


#### add new delta_start_invoice, delta_index and consommation_sum to combined_df

In [40]:
combined_df['delta_index'] = combined_df['new_index'] - combined_df['old_index']
combined_df['delta_start_invoice'] = dates_df['delta_start_invoice']
combined_df['consommation_sum'] = sum([combined_df['consommation_level_1'], combined_df['consommation_level_2'], combined_df['consommation_level_3'], combined_df['consommation_level_4']])

#### new dataframe for one-hot encoding categorical variables (dis, catg, region, tarif_type, counter_statue, counter_code, reading_remarque, counter_type)

In [78]:
categorical_vars = ['dis', 'catg', 'region', 'tarif_type', 'counter_statue', 'counter_code', 'reading_remarque', 'counter_type']
categorical_df = pd.get_dummies(combined_df, columns=categorical_vars, prefix=categorical_vars)
categorical_df = categorical_df.groupby('id').agg({col: 'max' for col in categorical_df.columns if col != 'id'})



#### Agg function to group the transactions with each client

In [79]:
stats = ['sum', 'mean', 'max', 'min', 'median', 'std']

# Select only the columns you want to aggregate
selected_columns = ['counter_number', 'counter_coefficient', 'consommation_level_1', 
                    'consommation_level_2', 'consommation_level_3', 'consommation_level_4',
                    'consommation_sum', 'delta_index', 'delta_start_invoice', 'id']

# Create a new dataframe with the desired aggregate functions
numerical_df = combined_df[selected_columns].groupby('id').agg(stats)

numerical_df.head()




counter_number                                              \
                             sum       mean        max        min     median   
id                                                                             
train_Client_0        46748345.0  1335667.0  1335667.0  1335667.0  1335667.0   
train_Client_1        25119374.0   678902.0   678902.0   678902.0   678902.0   
train_Client_10       10309770.0   572765.0   572765.0   572765.0   572765.0   
train_Client_100         41560.0     2078.0     2078.0     2078.0     2078.0   
train_Client_1000       274050.0    19575.0    19575.0    19575.0    19575.0   

                       counter_coefficient               ... delta_index       \
                   std                 sum mean max min  ...         max  min   
id                                                       ...                    
train_Client_0     0.0                  35  1.0   1   1  ...        1386   38   
train_Client_1     0.0                  37  1.0   1   1  ...        1207  190   
train_Client_10    0.0                  18  1.0   1   1  ...        3082  188   
train_Client_100   0.0                  20  1.0   1   1  ...          15    0   
train_Client_1000  0.0                  14  1.0   1   1  ...        2382  124   

                                     delta_start_invoice                      \
                  median         std                 sum         mean    max   
id                                                                             
train_Client_0     267.0  341.553930              213142  6089.771429   8844   
train_Client_1     520.0  197.935960              132603  3583.864865   6152   
train_Client_10    655.5  646.808386              165982  9221.222222  12103   
train_Client_100     0.0    3.607011               91275  4563.750000   5920   
train_Client_1000  770.0  633.485669               13497   964.071429   1707   

                                              
                    min  median          std  
id                                            
train_Client_0     3943  6047.0  1358.574709  
train_Client_1     1239  3509.0  1457.748762  
train_Client_10    7182  8678.0  1526.789733  
train_Client_100   3256  4545.5   774.520692  
train_Client_1000   122  1010.0   506.611437  

[5 rows x 54 columns]

#### Combining numerical and cat dataframes

In [86]:
to_drop = ['region', 'date_x', 'dis', 'id', 'catg', 'target', 'date_y', 'tarif_type', 'counter_number', 
           'counter_statue', 'counter_code', 'reading_remarque', 'consommation_level_4', 'old_index',
           'new_index', 'months_number', 'counter_type', 'counter_coefficient', 'consommation_level_1',
           'consommation_level_2', 'consommation_level_3']

client_summary = pd.concat([numerical_df, categorical_df], axis=1)

# Identify existing columns in the DataFrame
existing_columns = [col for col in to_drop if col in client_summary.columns]

# Drop existing columns from the DataFrame
client_summary = client_summary.drop(columns=existing_columns)

client_summary.info()




<class 'pandas.core.frame.DataFrame'>
Index: 21652 entries, train_Client_0 to train_Client_128438
Columns: 146 entries, ('counter_number', 'sum') to counter_type_GAZ
dtypes: bool(89), float64(30), int64(27)
memory usage: 11.4+ MB


#### Add y variable

In [87]:
client_summary['target'] = combined_df.groupby('id')['target'].apply(lambda x: 1 if x.any() else 0)
client_summary['target'].value_counts()


target
0    20576
1     1076
Name: count, dtype: int64

In [88]:
combined_df.groupby('id')['target'].apply(lambda x: 1 if x.any() else 0).value_counts()

target
0    20576
1     1076
Name: count, dtype: int64

## Data Balancing

#### Due to the low proportion of fraud cases, we performed synthetic oversampling of fraud cases with SMOTE and undersampled non-fraud cases with Tomek's link 

In [112]:
# Define the oversampling strategy using SMOTE
smote = SMOTE(sampling_strategy='auto')

smote = SMOTE(random_state=seed)
tomek = TomekLinks()

X = client_summary.drop('target', axis=1)

# Flatten multi-level column names
X.columns = [''.join(map(str, col)).strip() for col in X.columns.to_flat_index()]

y = client_summary['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= seed)

X_train, y_train = tomek.fit_resample(X_train, y_train)

X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

146


In [121]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)
data = np.column_stack((X, y))
df = pd.DataFrame(data)
column_series = df[146]
comparison_series = df.columns

spearman_corrs = []
p_values = []
for column in df.columns:
    if column != 146:  # Skip column 146 to avoid correlating with itself
        spearman_corr, p_value = spearmanr(column_series, df[column])
        spearman_corrs.append(spearman_corr)
        p_values.append(p_value)

print("Spearman correlations and p-values between column 146 and other columns:")
for i, (corr, p_value) in enumerate(zip(spearman_corrs, p_values)):
    print(f"Column {i}: Spearman correlation = {corr:.4f}, p-value = {p_value:.4f}")



Spearman correlations and p-values between column 146 and other columns:
Column 0: Spearman correlation = 0.1064, p-value = 0.0000
Column 1: Spearman correlation = 0.0767, p-value = 0.0000
Column 2: Spearman correlation = 0.0932, p-value = 0.0000
Column 3: Spearman correlation = -0.0425, p-value = 0.0000
Column 4: Spearman correlation = -0.0046, p-value = 0.5063
Column 5: Spearman correlation = 0.1705, p-value = 0.0000
Column 6: Spearman correlation = 0.1035, p-value = 0.0000
Column 7: Spearman correlation = -0.0027, p-value = 0.6892
Column 8: Spearman correlation = -0.0027, p-value = 0.6892
Column 9: Spearman correlation = -0.0022, p-value = 0.7440
Column 10: Spearman correlation = -0.0022, p-value = 0.7440
Column 11: Spearman correlation = -0.0016, p-value = 0.8174
Column 12: Spearman correlation = 0.1213, p-value = 0.0000
Column 13: Spearman correlation = 0.0665, p-value = 0.0000
Column 14: Spearman correlation = 0.1062, p-value = 0.0000
Column 15: Spearman correlation = -0.0667, p-

hamming, 